# <center> Hoja de trabajo

In [20]:
#!pip install pymysql

In [1]:
import math
import dash          
import dash_bootstrap_components as dbc
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
from sqlalchemy import create_engine
import requests
import random
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import sys
import pymysql

In [134]:
# base producción agrícola
df_produccion = pd.read_excel('C:/Users/jcmartinez/Desktop/Dashboard_SEGALMEX/datasets/base_prodAgricola_con_claves_inegi.xlsx')
df_produccion = df_produccion.dropna()
#df_produccion = df_produccion[df_produccion['Anio']==2020]
df_produccion = df_produccion[['Anio','CVE_ENT','NOM_ENT','CVE_MUN','NOM_MUN','LAT_DECIMAL','LON_DECIMAL','Volumenproduccion']]
df_produccion.head()

,Anio,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,LAT_DECIMAL,LON_DECIMAL,Volumenproduccion
0,2019,1,Aguascalientes,1,Aguascalientes,21.88,-102.3,41182.48
1,2019,1,Aguascalientes,1,Aguascalientes,21.88,-102.3,945.00
2,2019,1,Aguascalientes,1,Aguascalientes,21.88,-102.3,12037.00
3,2019,1,Aguascalientes,1,Aguascalientes,21.88,-102.3,22808.00
4,2019,1,Aguascalientes,1,Aguascalientes,21.88,-102.3,4013.60


In [135]:
estados_porcentaje_color = df_produccion.groupby(['Anio', 'NOM_ENT'])['Volumenproduccion'].sum()
estados_porcentaje_color = estados_porcentaje_color.reset_index()
estados_porcentaje_color

,Anio,NOM_ENT,Volumenproduccion
0,2019,Aguascalientes,2451257.66
1,2019,Baja California,3137750.66
2,2019,Baja California Sur,845966.30
3,2019,Campeche,2218491.00
4,2019,Chiapas,8783440.86
...,...,...,...
91,2021,Tamaulipas,5715133.27
92,2021,Tlaxcala,1375547.41
93,2021,Veracruz de Ignacio de la Llave,28484382.10
94,2021,Yucatán,577746.95


In [138]:
estados_porcentaje_color.columns = ['Anio', 'Entidad', 'Valor']
estados_porcentaje_color.to_excel('../datasets/estados_produccion_color.xlsx', index=False)

In [65]:
maximos_por_anio = estados_porcentaje_color.groupby('Anio')['Volumenproduccion'].max()
maximos_por_anio.reset_index()

,Anio,Volumenproduccion
0,2019,2.527242e+08
1,2020,2.982553e+07
2,2021,2.857926e+07


In [123]:
def recode_volumen(anio, volumen):
    new_values = []
    anio = anio.values
    vol = volumen.values
    for anio, vol in zip(anio, vol):
        if anio == 2019:
            new_values.append(vol/1000000000)
        else:
            new_values.append(vol/100000000)
    
    return new_values

In [127]:
new_col = [vol/1000000000 if anio == 2019 else vol/100000000 for anio, vol in zip(estados_porcentaje_color['Anio'], estados_porcentaje_color['Volumenproduccion'])]
estados_porcentaje_color['Volumenproduccion2'] = new_col
estados_porcentaje_color.head()

,Anio,NOM_ENT,Volumenproduccion,Volumenproduccion2
0,2019,Aguascalientes,2451257.66,0.002451
1,2019,Baja California,3137750.66,0.003138
2,2019,Baja California Sur,845966.30,0.000846
3,2019,Campeche,2218491.00,0.002218
4,2019,Chiapas,8783440.86,0.008783


In [128]:
maximos_por_anio = estados_porcentaje_color.groupby('Anio')['Volumenproduccion2'].max()
maximos_por_anio.reset_index()

,Anio,Volumenproduccion2
0,2019,0.252724
1,2020,0.298255
2,2021,0.285793


In [129]:
estados_porcentaje_color = estados_porcentaje_color.drop('Volumenproduccion', axis=1)
estados_porcentaje_color.columns = ['Anio',	'NOM_ENT', 'porcentaje']
estados_porcentaje_color.head()

,Anio,NOM_ENT,porcentaje
0,2019,Aguascalientes,0.002451
1,2019,Baja California,0.003138
2,2019,Baja California Sur,0.000846
3,2019,Campeche,0.002218
4,2019,Chiapas,0.008783


In [ ]:
estados_porcentaje_color.to_excel('../datasets/estados_produccion_color.xlsx', index=False)

In [22]:
# urls
repo_est_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' 
repo_mun_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/MunicipiosMexico.json'


In [26]:
# read 
mx_est_geo = requests.get(repo_est_url).json()
mx_mun_geo = requests.get(repo_mun_url).json()

In [35]:
# estados porcentaje
df_estados = pd.read_excel('C:/Users/jcmartinez/Desktop/Dashboard_SEGALMEX/datasets/df_estados_porcentaje.xlsx',  usecols='B:C')
df_estados['percentage'] = df_estado['percentage']/10000000

,estado,percentage
0,Oaxaca,0.345588
1,Nuevo león,0.333333
2,Guerrero,0.321839
3,Hidalgo,0.308571
4,Baja california sur,0.301205


In [24]:
    fig = go.Figure()

    # Traza areas de producción
    fig.add_trace(go.Choroplethmapbox(name='Mexico', geojson=mx_est_geo, ids=df_estados['estado'], z=df_estados['percentage'],
                                    locations=df_estados['estado'], featureidkey='properties.name', colorscale='greens',
                                    marker=dict(line=dict(color='black'), opacity=0.2)))

    # Traza de centros de acopio
    fig.add_trace(go.Scattermapbox(
            lat=df_centros['LAT_DECIMAL'],
            lon=df_centros['LON_DECIMAL'],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=5
            ),
            text=df_centros['NOM_MUN'],
        ))
    # Traza de beneficiarios 
    fig.add_trace(go.Scattermapbox(
            lat=benef_filter['Lat'],
            lon=benef_filter['Lon'],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=8
            ),
            text=benef_filter['Valor'],
        ))
    
    fig.update_layout(mapbox_style='open-street-map',
                    mapbox_zoom=4, 
                    mapbox_center = {'lat': 25, 'lon': -99}
                    )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

NameError: name 'df_estados' is not defined